In [1]:
# py-sc-fermi

import timeit

start = timeit.default_timer()

from py_sc_fermi.defect_charge_state import DefectChargeState
from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.inputs import inputs_from_files

import numpy as np

import sc_pyper_functions as scpf
import pandas as pd


defects = scpf.import_calculations_from_file('defects.yaml')
elements = scpf.import_calculations_from_file('elements.yaml')
interest = scpf.import_calculations_from_file('interest.yaml')
grid = (pd.read_csv('grid.dat', skiprows=10, sep='\s+'))  # 'grid' refers to a stability region divded into a grid of approximatley 80 points
ex_grid = (pd.read_csv('ex_grid.dat', skiprows=10, sep='\s+'))

import yaml

with open('automator_config.yaml', 'r') as stream:
    try:
        a = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

def make_all_defects(config_dict,chem_pots):
    to_write = []
    defs = []
    for k,v in config_dict['defects'].items():
        label = [i for i in config_dict['defects'][k][1:]]
        to_calcs = [defects[i] for i in label]
        out = scpf.make_defect(to_calcs, elements, interest['LLZO'], delta_mu=chem_pots, corr=config_dict['iccs'], sites=1)
        out.n_sites = config_dict['defects'][out.label][0]
        #print(out)
        defs.append(out)
    return defs


unitcell_filename = 'unitcell.dat'
totdos_filename = 'totdos.dat'
input_fermi_filename = 'input-fermi.dat'

inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                           input_fermi_filename=input_fermi_filename, 
                           totdos_filename=totdos_filename)

examples =  (75,12,42)  

tls = []
ex = []
for i in examples:
    b = make_all_defects(a,grid.iloc[i].to_dict())

    scf = scpf.SCFermi( b, a['nelect'], a['e_gap'], 1500, a['spin_polarised'])
    scf.output()
    
    defect_system = DefectSystem( defect_species=inputs['defect_species'],
                                  volume=inputs['volume'],
                                  dos=inputs['dos'],
                                  temperature=inputs['temperature'])

    #print(defect_system)

    unitcell_filename = 'unitcell.dat'
    totdos_filename = 'totdos.dat'
    input_fermi_filename = 'input-fermi.dat'

    inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                                   input_fermi_filename=input_fermi_filename, 
                                   totdos_filename=totdos_filename)
    
    out = defect_system.to_dict()
    out_tls = defect_system.get_transition_levels()
    ex.append(out)
    tls.append(out_tls)
          
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(2,3, figsize =(32,18))
axs=axs.flatten()
for j,i in enumerate(ex):
    ax=axs[j]
    for k,v in i.items():
        if k != 'Fermi Energy':
            ax.bar(k,np.log10(v))

for j,i in enumerate(tls):
    ax=axs[j+3]
    for tl in i.values():
        ax.plot(tl[0],tl[1], '-o', label=i)
    ax.set_xlim(0,5.904)
    ax.set_ylim(0,8)
plt.show()

ModuleNotFoundError: No module named 'sc_pyper_functions'

In [4]:
from py_sc_fermi.defect_charge_state import FrozenDefectChargeState

# example_dir = 'example_inputs'

# unitcell_filename = 'unitcell.dat'
# totdos_filename = 'totdos.dat'
# input_fermi_filename = 'input-fermi.dat'

# # TODO: Update the inputs.read_input_data() function to handle frozen-sc-fermi input files.
# inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
#                            input_fermi_filename=input_fermi_filename, 
#                            totdos_filename=totdos_filename)

# defect_system = DefectSystem( defect_species=inputs['defect_species'],
#                               volume=inputs['volume'],
#                               dos=inputs['dos'], # py_sc_fermi.DOS object,
#                               temperature=300.0 )

# defect_system

# from collections import Counter
# nfrozen = 1
# namef = ['v_Li']
# concf = [0.3285677364522e20]
# # check frozen defects are included in the original list of defects
# for name in namef:
#     if name not in defect_system.defect_species_names:
#         raise ValueError((f'ERROR: fixed concentration defect "{name}"'+
#                            'is not in list of original defects!\n'+
#                            'Please add new frozen defects as fixed charge defects in'
#                            'a separate list - otherwise a solution cannot be found!!'))
# # Check if any defects have been assigned multiple fixed concentrations
# c = Counter(namef)
# for n, count in c.items():
#     if count > 1:
#         raise ValueError(f'ERROR: defect "{n}"has multiple fixed concentrations!!')
# for n, c in zip(namef, concf):
#     defect_system.defect_species_by_name(n).fix_concentration( c * defect_system.volume / 1e24 )
    
# defect_system.report()

ficharg = False
fixconch = False
namefn = ['V_Ga', 'Ga_i']
chargefn = [-1.0, 1.0]
concfn = [0.19e19, 0.5e20]
if 0 in chargefn:
    checkfixchg = True
else:
    checkfixchg = False
for n,q,c  in zip( namefn, chargefn, concfn ):
    print(c)
    c = c * defect_system.volume / 1e24
    frozen_charge_state = FrozenDefectChargeState(charge=q, concentration=c)
    if n in defect_system.defect_species_names:     
        defect_system.defect_species_by_name(n).charge_states[q] = frozen_charge_state
    else:
        ds = DefectSpecies(name=n, nsites=1, charge_states=[frozen_charge_state])
        defect_system.defect_species.append(ds)

defect_system.check_concentrations()
defect_system.to_dict()

1.9e+18
5e+19


/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1763: RuntimeWarning: invalid value encountered in double_scalars
  q = 2.0 * (q - r)


{'Fermi Energy': 2.268876135271157,
 'Ga_i': 5e+19,
 'La_Zr': 1138600187751409.5,
 'Li_La': 4.4451229991332216e+16,
 'Li_Zr': 2.137927217473788e+16,
 'Li_i': 1.576011598661271e+19,
 'O_i': 16536344102331.236,
 'V_Ga': 1.9e+18,
 'Zr_La': 2422751546208918.5,
 'Zr_Li': 168255551662315.53,
 'Zr_Li_tet': 1780158.0407072108,
 'Zr_i': 53806154.49389492,
 'n0': 238043947855379.97,
 'p0': 238043947855379.97,
 'v_La': 4597859133442.098,
 'v_Li': 6.370925377113671e+19,
 'v_O': 10069448064254.527,
 'v_Zr': 4097712.530548102}

In [36]:
from pandas import DataFrame as df
from itertools import product
from tqdm import tqdm
from scipy.constants import physical_constants

kb_e = physical_constants['Boltzmann constant in eV/K'][0]
j_to_ev = 1/physical_constants['electron volt-joule relationship'][0]
S_0 = 205 * j_to_ev / physical_constants['Avogadro constant'][0]
Cp = (7/2)*kb_e

temperature = np.arange(800, 1600, 100)
pressure = np.logspace(-1, -10, num=10)
in_conds = product(pressure,temperature)

def dependance(P,T):
    """
    
    This function gives dependance of mu_O(T,P)
    args: P = pressure (float)
          T = temperature (float)
          
    returning:
          mu_O(T,P) (float)
          
    assuming oxygen is an ideal gas
    
    """
    
    chem_pot = 0.5 * ( (Cp * (T - 298))
                      - T * ( (S_0 + (Cp * np.log(T/298)) + (kb_e * np.log((0.21/P)) ) ) ))  
    return chem_pot                        

out_undoped=[]
for i in tqdm(in_conds):
    mu_o = dependance(i[0], i[1])    # calculate oxygen chemical potential based of temperature and pressure 
    for j in range(ex_grid.shape[0]):
        if round(mu_o,1) == round(ex_grid['mu_O'][j],1):
                b = make_all_defects(a,ex_grid.iloc[j].to_dict())

                scf = scpf.SCFermi( b, a['nelect'], a['e_gap'], 1500, a['spin_polarised'])
                scf.output()

                defect_system = DefectSystem( defect_species=inputs['defect_species'],
                                              volume=inputs['volume'],
                                              dos=inputs['dos'],
                                              temperature=inputs['temperature'])
                out_undoped.append(defect_system.to_dict())

            
#pd.DataFrame(out_undoped, columns=('P', 'T', 'ind', 'mu_o','electrons','holes','efermi')).to_csv('undoped')

0it [00:00, ?it/s]


NameError: name 'scpf' is not defined

In [25]:
from auto_functions import *
import yaml
from py_sc_fermi.inputs import inputs_from_files
from pandas import DataFrame as df
import pandas as pd
from itertools import product
from tqdm import tqdm
import numpy as np
from scipy.constants import physical_constants

with open('automator_config.yaml', 'r') as stream:
    try:
        a = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

defects = import_calculations_from_file('defects.yaml')
elements = import_calculations_from_file('elements.yaml')
interest = import_calculations_from_file('interest.yaml')
        

def make_all_defects(config_dict,chem_pots):
    to_write = []
    defs = []
    for k,v in config_dict['defects'].items():
        label = [i for i in config_dict['defects'][k][1:]]
        to_calcs = [defects[i] for i in label]
        out = make_defect(to_calcs, elements, interest['LLZO'], delta_mu=chem_pots, corr=config_dict['iccs'], sites=1)
        out._nsites = config_dict['defects'][out.name][0]
        #print(out)
        defs.append(out)
    return defs

ex_grid = (pd.read_csv('ex_grid.dat', skiprows=10, sep='\s+'))
grid = (pd.read_csv('grid.dat', skiprows=10, sep='\s+'))
all_defects = make_all_defects(a,ex_grid.iloc[5497].to_dict())
#print(b)

In [26]:
grid = (pd.read_csv('grid.dat', skiprows=10, sep='\s+'))  # 'grid' refers to a stability region divded into a grid of approximatley 80 points
ex_grid = (pd.read_csv('ex_grid.dat', skiprows=10, sep='\s+'))



unitcell_filename = 'unitcell.dat'
totdos_filename = 'totdos.dat'
input_fermi_filename = 'input-fermi.dat'

inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                           input_fermi_filename=input_fermi_filename, 
                           totdos_filename=totdos_filename)


defect_system_unrelaxed = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=1500)

#defect_system.report()
constraint = (defect_system_unrelaxed.to_dict()['v_Li']) - (defect_system_unrelaxed.to_dict()['Li_i'])

defect_system_unrelaxed.report()

Volume of cell: 1056.622886347127 A^3
Found spin polarised system...
Number of electrons in system: 544
Energy gap of system: 5.904 eV
Temperature: 1500.0 K
Number of defect species: 13
Reading spin-polarised DOS
SC Fermi level :      3.378588089859929  (eV)

Concentrations:
n (electrons)  : 4847943608420.384 cm^-3
p (holes)      : 44482920693.26799 cm^-3
v_O            : 86607208011835.23 cm^-3
v_Li           : 3.168454505025838e+19 cm^-3
Li_i           : 3.1689368227470954e+19 cm^-3
O_i            : 104811.65773031449 cm^-3
v_La           : 223517456786.10812 cm^-3
v_Zr           : 157591.95201718222 cm^-3
Zr_i           : 1401348029.2747295 cm^-3
Zr_Li_tet      : 123190077990.1366 cm^-3
Zr_Li          : 2176065384489526.8 cm^-3
Zr_La          : 3062494056181169.0 cm^-3
La_Zr          : 900328195929401.9 cm^-3
Li_La          : 4344991936475145.0 cm^-3
Li_Zr          : 1652075880227931.8 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1763: RuntimeWarning: invalid value encountered in double_scalars
  q = 2.0 * (q - r)


In [27]:
#print(b)

defect_system_full_relax = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300)

# defect_system_sunghyun.defect_species_by_name('v_Li').fix_concentration(defect_system_unrelaxed.to_dict()['v_Li'] / 1e24 * defect_system_sunghyun.volume)
# defect_system_sunghyun.defect_species_by_name('Li_i').fix_concentration(defect_system_unrelaxed.to_dict()['Li_i'] / 1e24 * defect_system_sunghyun.volume)
# #defect_system_sunghyun.get_constrained_sc_fermi({'v_Li': +1, 'Li_i': -1}, constraint)
defect_system_full_relax.report()
#defect_system_full_relax

SC Fermi level :      3.435400610912699  (eV)

Concentrations:
n (electrons)  : 1.410777787112445e-21 cm^-3
p (holes)      : 2.8920215984735876e-37 cm^-3
v_O            : 1.0282294020879069e-19 cm^-3
v_Li           : 4424250209729.337 cm^-3
Li_i           : 4424258787811.561 cm^-3
O_i            : 1.5766234074935817e-59 cm^-3
v_La           : 3.1720433505619897e-26 cm^-3
v_Zr           : 7.961214104162664e-55 cm^-3
Zr_i           : 1.0168276604181081e-42 cm^-3
Zr_Li_tet      : 2.7266172353264954e-32 cm^-3
Zr_Li          : 5.2053723899453675e-12 cm^-3
Zr_La          : 2.330844971766325e-09 cm^-3
La_Zr          : 4.1492671933788866e-10 cm^-3
Li_La          : 9.779563656522404e-06 cm^-3
Li_Zr          : 1.1195706995312348e-05 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_O        : Charge Concentration(cm^-3) Total
           :  2  9.969931e-20          96.96 
           :  1  3.123630e-21          3.04 
      

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1760: RuntimeWarning: invalid value encountered in double_scalars
  r = (xf - nfc) * (fx - ffulc)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy

In [28]:
from py_sc_fermi.defect_charge_state import DefectChargeState, FrozenDefectChargeState
from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.inputs import inputs_from_files

all_frozen_charge_states = {}
for i in all_defects:
    frozen_charge_states = {}
    for j in i.charge_states:
        c = i.charge_states[j].get_concentration(defect_system_unrelaxed.get_sc_fermi(verbose=False), defect_system_unrelaxed.temperature)
        frozen_charge_state = FrozenDefectChargeState(charge= j , concentration=c)
        frozen_charge_states.update({j : frozen_charge_state})
    all_frozen_charge_states.update({i.name:frozen_charge_states})
for x in all_defects:
     x._charge_states = None
     x._charge_states = all_frozen_charge_states[x.name]
print(all_defects)

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1763: RuntimeWarning: invalid value encountered in double_scalars
  q = 2.0 * (q - r)


[
v_O, nsites=3
  q=+2, [c]=1.95371445701975e-08
  q=+1, [c]=1.0964709087204011e-08
  q=+0, [c]=1.865711908943459e-12
, 
v_Li, nsites=3
  q=+0, [c]=4.343058814158191e-12
  q=-1, [c]=0.011159538476856802
, 
Li_i, nsites=1
  q=+0, [c]=2.5706587612672673e-09
  q=+1, [c]=0.033483709152368535
, 
O_i, nsites=1
  q=+0, [c]=1.1074639464890854e-16
  q=-1, [c]=1.664923532741314e-24
, 
v_La, nsites=2
  q=-3, [c]=1.180868301691534e-10
, 
v_Zr, nsites=1
  q=-4, [c]=1.6651526320547301e-16
, 
Zr_i, nsites=1
  q=+0, [c]=1.2745963527569528e-35
  q=+1, [c]=3.75307978035854e-30
  q=+2, [c]=1.1215972007106742e-19
  q=+3, [c]=2.4074817641882648e-15
  q=+4, [c]=1.4782888055452145e-12
, 
Zr_Li_tet, nsites=1
  q=+0, [c]=5.020959929938431e-24
  q=+1, [c]=4.474211205277478e-19
  q=+2, [c]=1.301654553278397e-10
  q=+3, [c]=3.316177320445699e-67
, 
Zr_Li, nsites=2
  q=+0, [c]=7.946999107003572e-21
  q=+1, [c]=2.891016063533735e-15
  q=+2, [c]=1.3031480792641396e-10
  q=+3, [c]=1.1495099260207468e-06
, 
Zr_La, nsi

In [13]:
b[1] = make_defect([defects['v_Li'],defects['v_Li_-']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=3)
b[2] = make_defect([defects['Li_i'],defects['Li_i_+']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=1)

#print(b)

defect_system_full_relax = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300)

# defect_system_sunghyun.defect_species_by_name('v_Li').fix_concentration(defect_system_unrelaxed.to_dict()['v_Li'] / 1e24 * defect_system_sunghyun.volume)
# defect_system_sunghyun.defect_species_by_name('Li_i').fix_concentration(defect_system_unrelaxed.to_dict()['Li_i'] / 1e24 * defect_system_sunghyun.volume)
# #defect_system_sunghyun.get_constrained_sc_fermi({'v_Li': +1, 'Li_i': -1}, constraint)
defect_system_full_relax.report()
#defect_system_full_relax

SC Fermi level :      3.2546360224006716  (eV)

Concentrations:
n (electrons)  : 1.2964345955216192e-24 cm^-3
p (holes)      : 3.147092684096708e-34 cm^-3
v_O            : 86607208011835.23 cm^-3
v_Li           : 4065665821.74269 cm^-3
Li_i           : 4814470428236541.0 cm^-3
O_i            : 104811.65773031449 cm^-3
v_La           : 223517456786.10812 cm^-3
v_Zr           : 157591.95201718222 cm^-3
Zr_i           : 1401348029.2747295 cm^-3
Zr_Li_tet      : 123190077990.1366 cm^-3
Zr_Li          : 2176065384489526.8 cm^-3
Zr_La          : 3062494056181169.0 cm^-3
La_Zr          : 900328195929401.9 cm^-3
Li_La          : 4344991936475145.0 cm^-3
Li_Zr          : 1652075880227931.8 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_O        : Charge Concentration(cm^-3) Total
           :  2  5.547053e+13          64.05  [fixed]
           :  1  3.113138e+13          35.95  [fixed]
           :  0  5.297193e+09   

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-package

In [35]:
all_defects[1] = make_defect([defects['v_Li'],defects['v_Li_-']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=3)
all_defects[2] = make_defect([defects['Li_i'],defects['Li_i_+']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=1)

defect_system_sunghyun = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=600)

defect_system_sunghyun.defect_species_by_name('v_Li').fix_concentration(defect_system_unrelaxed.to_dict()['v_Li'] / 1e24 * defect_system_sunghyun.volume)
defect_system_sunghyun.defect_species_by_name('Li_i').fix_concentration(defect_system_unrelaxed.to_dict()['Li_i'] / 1e24 * defect_system_sunghyun.volume)
# #defect_system_sunghyun.get_constrained_sc_fermi({'v_Li': +1, 'Li_i': -1}, constraint)
defect_system_sunghyun.report()

SC Fermi level :      4.720639640863127  (eV)

Concentrations:
n (electrons)  : 70486802348.70155 cm^-3
p (holes)      : 7.142145744540517e-19 cm^-3
v_O            : 86607208011835.23 cm^-3
v_Li           : 3.1684544751421088e+19 cm^-3
Li_i           : 3.168936271464413e+19 cm^-3
O_i            : 104811.65773031449 cm^-3
v_La           : 223517456786.10812 cm^-3
v_Zr           : 157591.95201718222 cm^-3
Zr_i           : 1401348029.2747295 cm^-3
Zr_Li_tet      : 123190077990.1366 cm^-3
Zr_Li          : 2176065384489526.8 cm^-3
Zr_La          : 3062494056181169.0 cm^-3
La_Zr          : 900328195929401.9 cm^-3
Li_La          : 4344991936475145.0 cm^-3
Li_Zr          : 1652075880227931.8 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_O        : Charge Concentration(cm^-3) Total
           :  2  5.547053e+13          64.05  [fixed]
           :  1  3.113138e+13          35.95  [fixed]
           :  0  5.297193e+09

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))


In [44]:
all_defects[1] = make_defect([defects['v_Li'],defects['v_Li_-']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=3)
all_defects[2] = make_defect([defects['Li_i'],defects['Li_i_+']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=1)

defect_system_ben = DefectSystem( defect_species=all_defects,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=00)

defect_system_ben.get_constrained_sc_fermi({'v_Li': +1, 'Li_i': -1}, constraint)
#defect_system_ben

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)


3.3785880877259333

In [10]:
from auto_functions import *
from py_sc_fermi.inputs import inputs_from_files
from pandas import DataFrame as df
import pandas as pd
from itertools import product
from tqdm import tqdm
import numpy as np
from scipy.constants import physical_constants

grid = (pd.read_csv('grid.dat', skiprows=10, sep='\s+'))  # 'grid' refers to a stability region divded into a grid of approximatley 80 points
ex_grid = (pd.read_csv('ex_grid.dat', skiprows=10, sep='\s+'))

def dependance(P,T):
    """
    
    This function gives dependance of mu_O(T,P)
    args: P = pressure (float)
          T = temperature (float)
          
    returning:
          mu_O(T,P) (float)
          
    assuming oxygen is an ideal gas
    
    """
    
    chem_pot = 0.5 * ( (Cp * (T - 298))
                      - T * ( (S_0 + (Cp * np.log(T/298)) + (kb_e * np.log((0.21/P)) ) ) ))  
    return chem_pot          

kb_e = physical_constants['Boltzmann constant in eV/K'][0]
j_to_ev = 1/physical_constants['electron volt-joule relationship'][0]
S_0 = 205 * j_to_ev / physical_constants['Avogadro constant'][0]
Cp = (7/2)*kb_e

temperature = np.arange(800, 1600, 100)
pressure = np.logspace(-1, -10, num=10)
in_conds = product(pressure,temperature)

unitcell_filename = 'unitcell.dat'
totdos_filename = 'totdos.dat'
input_fermi_filename = 'input-fermi.dat'

# inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
#                            input_fermi_filename=input_fermi_filename, 
#                            totdos_filename=totdos_filename)

# out_undoped=[]
# for i in tqdm(in_conds):
#         mu_o = dependance(i[0], i[1])    # calculate oxygen chemical potential based of temperature and pressure 
#         for j in range(ex_grid.shape[0]):
#             if round(mu_o,1) == round(ex_grid['mu_O'][j],1):
#                     b = make_all_defects(a,ex_grid.iloc[j].to_dict())

defect_system = DefectSystem( defect_species=b,
                                volume=inputs['volume'],
                                dos=inputs['dos'],
                                temperature=300)
#         out_undoped.append(defect_system.to_dict())
defect_system.report()
#defect_system.get_constrained_sc_fermi({'v_Li': +1, 'Li_i': -1}, constraint)
print(defect_system.to_dict()['v_Li'] - defect_system.to_dict()['Li_i'])
print(constraint)
ahhh = defect_system.get_constrained_sc_fermi({'v_Li': +1, 'Li_i': -1}, constraint)


/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_species.py:221: RuntimeWarning: invalid value encountered in double_scalars
  cs_concentrations[q] *= scaling
/mnt/c/Users/Alex/Materials_work/new_fermi/p

SC Fermi level :      -27.577780507626215  (eV)

Concentrations:
n (electrons)  : 0.0 cm^-3
p (holes)      : 4.76915363262079e+23 cm^-3
v_O            : 86607208011835.23 cm^-3
v_Li           : 3.1684544751421088e+19 cm^-3
Li_i           : 3.168936271464413e+19 cm^-3
O_i            : 104811.65773031449 cm^-3
v_La           : 223517456786.10812 cm^-3
v_Zr           : 157591.95201718222 cm^-3
Zr_i           : 1401348029.2747295 cm^-3
Zr_Li_tet      : 123190077990.1366 cm^-3
Zr_Li          : 2176065384489526.8 cm^-3
Zr_La          : 3062494056181169.0 cm^-3
La_Zr          : 900328195929401.9 cm^-3
Li_La          : 4344991936475145.0 cm^-3
Li_Zr          : 1652075880227931.8 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_O        : Charge Concentration(cm^-3) Total
           :  2  5.547053e+13          64.05  [fixed]
           :  1  3.113138e+13          35.95  [fixed]
           :  0  5.297193e+09          0.0

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)
/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:1762: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/home/alex/anaconda3/lib/python3.6/site-package


v_O, nsites=3
  q=+2, [c]=1.95371445701975e-08
  q=+1, [c]=1.0964709087204011e-08
  q=+0, [c]=1.865711908943459e-12


v_Li, nsites=3
  q=+0, e=3.4713573800000104, deg=2
  q=-1, e=3.959671988940586, deg=1


Li_i, nsites=1
  q=+0, e=2.6462461100000683, deg=2
  q=+1, e=-2.939530639755103, deg=1


O_i, nsites=1
  q=+0, [c]=1.1074639464890854e-16
  q=-1, [c]=1.664923532741314e-24


v_La, nsites=2
  q=-3, [c]=1.180868301691534e-10


v_Zr, nsites=1
  q=-4, [c]=1.6651526320547301e-16


Zr_i, nsites=1
  q=+0, [c]=1.2745963527569528e-35
  q=+1, [c]=3.75307978035854e-30
  q=+2, [c]=1.1215972007106742e-19
  q=+3, [c]=2.4074817641882648e-15
  q=+4, [c]=1.4782888055452145e-12


Zr_Li_tet, nsites=1
  q=+0, [c]=5.020959929938431e-24
  q=+1, [c]=4.474211205277478e-19
  q=+2, [c]=1.301654553278397e-10
  q=+3, [c]=3.316177320445699e-67


Zr_Li, nsites=2
  q=+0, [c]=7.946999107003572e-21
  q=+1, [c]=2.891016063533735e-15
  q=+2, [c]=1.3031480792641396e-10
  q=+3, [c]=1.1495099260207468e-06


Zr_La, nsite

In [22]:
ficharg = False
fixconch = False
namefn = defect_system.defect_species_names
chargefn = [-1.0, 1.0]
concfn = [0.19e19, 0.5e20]
if 0 in chargefn:
    checkfixchg = True
else:
    checkfixchg = False
for n,q,c  in zip( namefn, chargefn, concfn ):
    print(c)
    c = c * defect_system.volume / 1e24

    if n in defect_system.defect_species_names:     
        defect_system.defect_species_by_name(n).charge_states[q] = frozen_charge_state
    else:
        ds = DefectSpecies(name=n, nsites=1, charge_states=[frozen_charge_state])
        defect_system.defect_species.append(ds)

1.9e+18
5e+19


In [35]:
for i,j in zip(all_frozen_charge_states,b):
    print(type(all_frozen_charge_states[j.name]))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [11]:
defect_system.defect_species_by_name('v_O').(all_frozen_charge_states['v_O'])

SyntaxError: invalid syntax (<ipython-input-11-3e7d8ae30852>, line 1)

In [41]:
b[0].charge_states

{0: q=+0, e=3.430977650000065, deg=1,
 1: q=+1, e=-0.9798363610594906, deg=2,
 2: q=+2, e=-4.522686024237847, deg=1}

In [17]:
b[1] = make_defect([defects['v_Li'],defects['v_Li_-']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=3)
b[2] = make_defect([defects['Li_i'],defects['Li_i_+']], elements, interest['LLZO'], delta_mu=ex_grid.iloc[5497].to_dict(), corr=a['iccs'], sites=1)

In [20]:
b

[
 v_O, nsites=3
   q=+2, [c]=1.95371445701975e-08
   q=+1, [c]=1.0964709087204011e-08
   q=+0, [c]=1.865711908943459e-12, 
 v_Li, nsites=3
   q=+0, e=3.4713573800000104, deg=2
   q=-1, e=3.959671988940586, deg=1, 
 Li_i, nsites=1
   q=+0, e=2.6462461100000683, deg=2
   q=+1, e=-2.939530639755103, deg=1, 
 O_i, nsites=1
   q=+0, [c]=1.1074639464890854e-16
   q=-1, [c]=1.664923532741314e-24, 
 v_La, nsites=2
   q=-3, [c]=1.180868301691534e-10, 
 v_Zr, nsites=1
   q=-4, [c]=1.6651526320547301e-16, 
 Zr_i, nsites=1
   q=+0, [c]=1.2745963527569528e-35
   q=+1, [c]=3.75307978035854e-30
   q=+2, [c]=1.1215972007106742e-19
   q=+3, [c]=2.4074817641882648e-15
   q=+4, [c]=1.4782888055452145e-12, 
 Zr_Li_tet, nsites=1
   q=+0, [c]=5.020959929938431e-24
   q=+1, [c]=4.474211205277478e-19
   q=+2, [c]=1.301654553278397e-10
   q=+3, [c]=3.316177320445699e-67, 
 Zr_Li, nsites=2
   q=+0, [c]=7.946999107003572e-21
   q=+1, [c]=2.891016063533735e-15
   q=+2, [c]=1.3031480792641396e-10
   q=+3, [c]=1.1

In [ ]:
defect_system.get_constrained_sc_fermi({'v_Li': +1, 'Li_i': -1}, constraint)

In [41]:
from py_sc_fermi.constants import kboltz
inputs['dos'].carrier_concentrations(ahhh, kboltz*400)[1] * 1e24 / defect_system.volume

/mnt/c/Users/Alex/Materials_work/new_fermi/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))


39848935.686796546

In [6]:
ahhh

3.4354006222315077

In [22]:
defect_system.to_dict()

/home/alex/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)


{'Fermi Energy': 10.01811012324589,
 'La_Zr': 900328195929401.9,
 'Li_La': 4344991936475145.0,
 'Li_Zr': 1652075880227931.8,
 'Li_i': 4.6765477785552375e+55,
 'O_i': 104811.65773031449,
 'Zr_La': 3062494056181169.0,
 'Zr_Li': 2176065384489526.8,
 'Zr_Li_tet': 123190077990.1366,
 'Zr_i': 1401348029.2747295,
 'n0': 2.117766400183517e+23,
 'p0': 2.1898093206929775e-12,
 'v_La': 223517456786.10812,
 'v_Li': 2.9986616001626077e+40,
 'v_O': 86607208011835.23,
 'v_Zr': 157591.95201718222}